# Metaclasses. OOP Patterns

## Rehearsal

`What is metaclasses?` is one of the most popular question on interviews.
A little back ago we talked that everything is an object in python, including classes.

```
class C:
   pass

C  # <class '__main__.C'>
type(C)  # <class "type" >

Metaclass is TYPE that creates instance of CLASS objects.

A = type('A', C.__bases__, dict(C.__dict__))
A  # <class '__main__.C'>
type(A)  # <class "type" >
```

- Metaclasses construct the whole instance of class when invoked
- exact steps:
   - creates a separate namespace
   - executes class body code in that namespace, it populates the namespace
   - creates instance of class object. class namespace becomes class.__dict__
- We can define our own metaclasses or derive them from type. 
- Since abovementioned steps are somewhat complicated, usually metaclasses are implemented as subclasses of type.
- Since we are now constructing class object ourselves, we have FULL CONTROL on how it will be created.
    - you can perform validation of function names. ->> module abc implements abstract classes.
    - you can add or remove additional class, static or instance methods automatically.
    - do whatever you want. This is one of the most powerful language feature in python.

In [2]:
class A:
    pass

class B:
    pass


class MyMeta(type):   # is a metaclass
    def __new__(cls, name, bases, dct):
        cls_instance = super().__new__(cls, name, bases, dct)
        def think(self):
            print('Cogito ergo sum')
        cls_instance.think = think   # adding instance method to ALL classes created with this metaclass
        return cls_instance

class C(A, B,  metaclass=MyMeta):    # A and B are BASES, using custom metaclass MyMeta
    pass

x = C()
x.think()

Cogito ergo sum


In [1]:

class DisallowPublicClassAttributes(type):  # is a metaclass
    def __new__(cls, name, bases, dct):
        cls_instance = super().__new__(cls, name, bases, dct)
        if any([not key.startswith("_") for key in dct.keys()]):
            raise Exception("Class `%s` can not have public attributes" % name)
        return cls_instance

class NoErrorClass(metaclass=DisallowPublicClassAttributes):
    __private = ""

class ErrorClass(metaclass=DisallowPublicClassAttributes):
    public = ""



Exception: Class `ErrorClass` can not have public attributes

# Section 1. OOP Patterns

  Three main categories:

    - Creational
        These patterns provide various object creation mechanisms,
        which increase flexibility and reuse of existing code.
    - Structural
        These patterns explain how to assemble objects and classes
        into larger structures while keeping these structures flexible and efficient.
    - Behavioral
        These patterns are concerned with algorithms and the assignment of
        responsibilities between objects.


## Creational patterns

    - Singleton
        Ensures that a class has just a single instance
    - Factory method
        Provides an interface for creating objects in
        a superclass, but allows subclasses to alter the
        type and behaviour of objects that will be created.

and others https://refactoring.guru/design-patterns/creational-patterns

    

### 3 most popular ways to implement singleton
Decorator

```python
def singleton(cls):
    instances = {}

    def getinstance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return getinstance

@singleton
class SingletonClass:
    pass
```

A base class

```python
class Singleton(object):
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not isinstance(cls._instance, type(cls)):
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance

class SingletonClass(Singleton):
    pass
```

A metaclass

```python
class Singleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]


class SingletonClass(metaclass=Singleton):
    pass
```

## Structural patterns

    - Decorator
        Lets you attach new behaviors to objects by placing these
        objects inside special wrapper objects that contain the behaviors.
    - Proxy
        Lets you provide a substitute or placeholder for another object.
        A proxy controls access to the original object, allowing you to perform
        something either before or after the request gets through to the
        original object.

and others https://refactoring.guru/design-patterns/structural-patterns

### Proxy pattern simple example

```python
class Product:
    def request(self) -> None:
        print("RealSubject: Handling request.")

class Proxy:
    def __init__(self, real_product: Product) -> None:
        self._real_product = real_product

    def request(self) -> None:
        if self.check_access():
            self._real_product.request()
            self.log_access()
        else:
            print("Forbidden")

    def check_access(self) -> bool:
        print("Proxy: Checking access prior to firing a real request.")
        return True

    def log_access(self) -> None:
        print("Proxy: Logging the time of request.", end="")
```

Python example:
    https://docs.python.org/3/library/types.html#types.MappingProxyType

## Behavioral patterns

    - Iterator
        Lets you traverse elements of a collection.
    - Observer
        Lets you define a subscription mechanism to notify multiple
        objects about any events that happen to the object they’re observing.

and others https://refactoring.guru/design-patterns/behavioral-patterns

### Observer simple example

```python
class Product:
    _observers = []
    def attach(self, observer):
        self._observers.append(observer)

    def notify(self) -> None:
        for observer in self._observers:
            observer.update(self)

    def do_some_logic(self) -> None:
        self.value = 2
        self.notify()


class ObserverA:
    def update(self, product):
        if product.value < 3:
            print("ObserverA: Reacted to the event")

product = Product()
observer_a = ObserverA()
product.attach(observer_a)
product.do_some_logic()
```